# This file initialize longtudinal plasma density profile (s,fs) and raw beam particles data $(x,p_x,y,p_y,\gamma)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from utils import *

%load_ext autoreload
%autoreload 2

plt.rcParams.update({'font.size': 20})

# Initialize longitudinal plasma density profile: s,fs

### Generate density upramp

In [ ]:
"""
# FACET-II parameters
L_ramp = 15000
n_entrance_over_n0 = 0.01
beta_m0 = 200 # Matched beta at the density plateau

s = np.linspace(0,L_ramp,15001)
fs = new_smooth_upramp(s,beta_m0,L_ramp,n_entrance_over_n0)
"""

# LC parameters
alpha_mi = 1
n_entrance_over_n0 = 0.0001
beta_m0 = np.sqrt(2 * 48786)

# s,fs,alpha_mi = smooth_upramp(beta_m0 = beta_m0,n_entrance_over_n0 = n_entrance_over_n0,alpha_mi=alpha_mi)
s,fs,alpha_mi = xinlu_upramp(beta_m0,n_entrance_over_n0,L_ramp=None,alpha_mi = alpha_mi)
    
"""
alpha_mi = 0.1
n_entrance_over_n0 = 0.01
beta_m0 = 200 # Matched beta at the density plateau

s,fs,alpha_mi = smooth_upramp(beta_m0 = beta_m0,n_entrance_over_n0 = n_entrance_over_n0,alpha_mi=alpha_mi)
"""
L_upramp = s[-1]
print('L_ramp =',L_upramp)

In [ ]:
plt.plot(s,fs)
plt.xlabel(r'$z\;\;(c/\omega_{p0})$')
plt.ylabel('$n(z)/n_0$')
plt.title('Plasma density profile')
plt.show()

### Trim the ramp

In [ ]:
s_cut = 2000

In [ ]:
# This function only keeps the portion where s <= s_cut
def trim(s,fs,s_cut):
    assert(len(s) == len(fs))
    if s_cut < s[0]:
        return np.array([]),np.array([])
    elif s_cut >= s[-1]:
        return s,fs
    for i in range(len(s)):
        if s[i] > s_cut:
            return s[:i],fs[:i]
s,fs = trim(s,fs,s_cut)
plt.plot(s,fs)
plt.xlabel(r'$z\;\;(c/\omega_{p0})$')
plt.ylabel('$n(z)/n_0$')
plt.title('Plasma density profile')
plt.show()

### Add a density downramp

In [ ]:
L_plateau = 20000
beta_m0_downramp = np.sqrt(1) * beta_m0

In [ ]:
n_exit_over_n0 = 0.0001
s_downramp,fs_downramp,alpha_mi = xinlu_upramp(beta_m0 = beta_m0_downramp,n_entrance_over_n0 = n_exit_over_n0,alpha_mi=alpha_mi)
s_downramp = s_downramp + L_upramp + L_plateau
fs_downramp = fs_downramp[::-1]


# # Add a symmetric downramp
s = np.append(s,s_downramp)
fs = np.append(fs,fs_downramp)

plt.plot(s,fs)
plt.xlabel(r'$z\;\;(c/\omega_{p0})$')
plt.ylabel('$n(z)/n_0$')
plt.title('Plasma density profile')
plt.show()

print('The total plasma length is:',s[-1])

### Add a uniform plasma region at the beginning and in the end

In [ ]:
L_uniform = 15000

In [ ]:
s = np.append(0,s+L_uniform)
s = np.append(s,s[-1]+L_uniform)
fs = np.append(fs[0],fs)
fs = np.append(fs,fs[-1])

plt.plot(s,fs)
plt.xlabel(r'$z\;\;(c/\omega_{p0})$')
plt.ylabel('$n(z)/n_0$')
plt.title('Plasma density profile')
plt.show()

### Load s,fs from QPAD input file

In [ ]:
qpinput_path = '../qpinput.json'
qpinput = load_json_file(qpinput_path)
s = qpinput['species'][0]['piecewise_s']
fs = qpinput['species'][0]['piecewise_fs']

### Manually setting s, fs

In [ ]:
s = [0,10000]
fs = [1,1]

plt.plot(s,fs)
plt.xlabel(r'$z\;\;(c/\omega_{p0})$')
plt.ylabel('$n(z)/n_0$')
plt.title('Plasma density profile')
plt.show()

### Save s,fs to input file

In [ ]:
save_s_fs(s,fs,path = 'sp_input.json')

### Sanity check: Plot s,fs from input file

In [ ]:
plot_s_fs_input(path = 'sp_input.json')

# Initialize beam particles' coordinates: $(x,p_x,y,p_y,\gamma)$

### Method 1: Initializing beam particles using Twiss parameters

In [ ]:
"""
# FACET-II parameters
alpha_i = 0.0
beta_i = 2000
emittance = 0.0594
gamma_mean = 20000
energy_spread = 0.02
N = 10000
x,px = Twiss_para_init(alpha_i,beta_i,emittance,gamma_mean,N)
gamma = np.random.normal(gamma_mean, energy_spread * gamma_mean, N)
"""

# LC parameters
alpha_i = 1
beta_i = 100 * beta_m0
emittance = 0.00594
gamma_mean = 48786
energy_spread = 0.0
N = 100
x,px = Twiss_para_init(alpha_i,beta_i,emittance,gamma_mean,N)
gamma = np.random.normal(gamma_mean, energy_spread * gamma_mean, N)

In [ ]:
# some auxiliary calculation
beta_m0 = np.sqrt(2 * gamma_mean) # matched beta at density plateau
beta_mi = beta_m0 / np.sqrt(n_entrance_over_n0)
sigma_mi = np.sqrt(beta_i * emittance / gamma_mean)

### Method 2: Initialize phase space rings
$$x = x_0 \sin(k_\beta z)$$
$$p_x = \gamma x' =  \gamma k_\beta x_0\cos(k_\beta z)$$

So:

$$\dfrac{p_0}{x_0} = \gamma k_\beta = \frac{\gamma}{\sqrt{2\gamma}}\frac{\omega_p}{c} = \sqrt{\frac{\gamma}{2}} \sqrt{\frac{n_{entrance}}{n_0}}\frac{\omega_{p0}}{c}$$

For $\gamma = 20000$, $n_{entrance}/n_0 = 0.01$, $p_0 / x_0 = 10 \omega_{p0} / c$

In [ ]:
x = np.array([])
px = np.array([])

x0 = sigma_mi
p0 = emittance / sigma_mi
N = 1000
multipliers = [0.1,0.3,0.5,1,2,3]

for multiplier in multipliers:
    x_ring,px_ring = phase_space_rings_init(x_max = x0 * multiplier,px_max = p0 * multiplier, N = N)
    x = np.hstack((x,x_ring))
    px = np.hstack((px,px_ring))

gamma = [gamma_mean] * len(x)

#### Optional: propagate in vacuum to transform rings to tilted

In [ ]:
gamma_i = (1 + alpha_i ** 2) / beta_i
z_relative_to_vacuum_focus = - alpha_i / gamma_i
x = x + px / gamma * z_relative_to_vacuum_focus

### Method 3: Manually initialize beam particles

In [ ]:
x = [0.001,0.002,0.005,0.01,0.02,0.05,0.1,0.2,0.5,1.0]
px = [0] * len(x)
gamma = [20000] * len(x)
y = [2]
py = [0]

### Save initial particle coordinates $(x,p_x,y,p_y,\gamma)$ to input file

In [ ]:
save_particle_coordinates(x,px,gamma,x,px)

### Sanity check: Plot beam particles' phase space from input file

In [ ]:
plot_init_phase_space(path = 'sp_input.json')

# Ion model (or custom focusing model) and acceleration model

### Set and save parameters to input file 
( input_file['plasma']['s'], input_file['acc_model']['s'], input_file['ion_model']['s'] must have the same first element and last element )

In [ ]:
input_file = load_json_file('sp_input.json')

input_file['simulation']['transverse_motion_model'] = "ion"


input_file['acc_model']['s'] = [0,s[-1]]
input_file['acc_model']['gamma_gain'] = [0,0]

input_file['ion_model']['type'] = "Gaussian"
input_file['ion_model']['s'] = [0,s[-1]]
input_file['ion_model']['A'] = [0,0]
input_file['ion_model']['sig_i'] = [1,1]
input_file['diag']['delta_s_dump'] = 100

save_to_json_file(input_file,'sp_input.json')

### Optional: Set acceleration model from QPAD result

In [ ]:
# Set model parameters from data analysis of QPAD simulation
parameters = load_json_file('../JupyterQpic/whole_beam/beam2')

s_acc_model = parameters['s']
gamma_QPAD = parameters['energy']
if s[-1] > s_acc_model[-1]:
    s_acc_model = s_acc_model + [s[-1]]
    gamma_QPAD = gamma_QPAD + [gamma_QPAD[-1]]

gamma_gain = np.array(gamma_QPAD) - gamma_QPAD[0]



input_file = load_json_file('sp_input.json')

input_file['acc_model']['s'] = list(s_acc_model)
input_file['acc_model']['gamma_gain'] = list(gamma_gain)

save_to_json_file(input_file,'sp_input.json')



# dic2 = load_json_file('../JupyterQpic/ion_density_transverse_lineout/ion_density_transverse_lineout_xi=0/ion_collapse_gaussian_fit.json')
# dic2.keys()

# s_ion_model = list(dic2['A_fit'].keys())
# A = [dic2['A_fit'][i] for i in s_ion_model]
# sig_i = [dic2['sigma_ion_fit'][i] for i in s_ion_model]
# s_ion_model = [int(i) for i in s_ion_model]
# s_ion_model = [0] + s_ion_model + [s[-1]]
# A = [0] + A + [A[-1]]
# sig_i = [sig_i[0]] + sig_i + [sig_i[-1]]

In [ ]:
plt.figure(1)
plt.plot(s_ion_model,A)

plt.figure(2)
plt.plot(s_ion_model,sig_i)

plt.figure(3)
plt.plot(s_acc_model,gamma_gain)
plt.show()

### Optional: Set focusing field model from QPAD result

In [ ]:
xi = 1.5
dt = 10
timeSteps = np.arange(0,7020,20)

In [ ]:
for i in range(len(timeSteps)):
        
    timeStep = timeSteps[i]

    r, Er = get_lineout(filename = '../Fields/Er/Re0/er_'+str(timeStep).zfill(8)+'.h5',\
                          direction = 'transverse',lineout_pos = xi,code = 'QPAD')
    _, Bphi = get_lineout(filename = '../Fields/Bphi/Re0/bphi_'+str(timeStep).zfill(8)+'.h5',\
                          direction = 'transverse',lineout_pos = xi,code = 'QPAD')
    assert(len(Er)==len(Bphi))
    Fr = Er - Bphi
    
    if i == 0:
        if s[-1] > timeSteps[-1] * dt:
            Fr_all = np.zeros((len(timeSteps)+1,len(Fr)))
        else:
            Fr_all = np.zeros((len(timeSteps),len(Fr)))
    Fr_all[i,:] = Fr

input_file = load_json_file('sp_input.json')

input_file['simulation']['transverse_motion_model'] = "focusing_field"
input_file['focusing_field_model'] = {}
input_file['focusing_field_model']['ff'] = Fr_all.tolist()
if s[-1] > timeSteps[-1] * dt:
    input_file['focusing_field_model']['s'] = (timeSteps * dt).tolist() + [s[-1]]
else:
    input_file['focusing_field_model']['s'] = (timeSteps * dt).tolist()
input_file['focusing_field_model']['r'] = r.tolist()

save_to_json_file(input_file,'sp_input.json')

### Sanity check: Plot the focusing field from 2d ($s,r$) interpolation 

In [ ]:
s_eval = 35000
r_max = 1

input_file = load_json_file('sp_input.json')

s_ff = np.array(input_file['focusing_field_model']['s'])
r_ff = np.array(input_file['focusing_field_model']['r'])

ff_ff = np.array(input_file['focusing_field_model']['ff'])

ff_model = interpolate.interp2d(s_ff, r_ff, ff_ff.T)


r = np.arange(0,r_max,0.001)
ff = ff_model([s_eval], r)
plt.plot(r,ff)
#plt.plot(r,r/2,'--')
plt.xlabel('$r$')
plt.ylabel('$F_r$')
plt.show()

In [ ]:
# s_idx = 300
# r_idx = 57

# print(ff_ff[s_idx][r_idx])

# s_eval = s_ff[s_idx]
# r_eval = [r_ff[r_idx]]
# ff_eval = ff_model(s_eval, r_eval)
# print(ff_eval)